# Summorize Recipient and create xlsx-Files

In [6]:
import sys
sys.path.insert(0, '../../data/lib/')

import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [7]:
#Const
out_path = '../../data/99. analyzes/'
YEAR = 2017
open_sheet = False

In [28]:
#Load Data
df_hcp = pd.read_csv('../../data/3. transformation/7_hcp_grouped_final.csv')
df_hco = pd.read_csv('../../data/3. transformation/7_hco_grouped_final.csv')

## Export addresses in groupes

In [29]:
def all_addresses(df, out):
    global open_sheet

    df = df.sort_values('group')
    df = df.set_index([df.group, df.index])
    df = df.sort_values(['group', 'main_address', 'name'])

    write_to_excel(df, out_path + 'excel/' + out + '.xlsx', open=open_sheet, index=True)
    df.to_csv(out_path + 'csv/' + out + '.csv', index=False)
    

#all_addresses(df_hcp.copy(), 'all_hcp_address_grouped')
all_addresses(df_hco.copy(), 'all_hco_address_grouped')

/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: 'group' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


## Aggregate

In [30]:
def aggregate_by_recipient(df, groupby, out, remcols):
    # Aggregiert nach Empfänger
    df.sort_values(by='main_address', inplace=True)

    grouped = df.groupby(by=groupby).agg({'name': 'first',
                                               'location': 'first',
                                               'address': 'first',
                                               'type': 'first',
                                               'year': 'first',
                                               'donations_grants': 'sum',
                                               'sponsorship': 'sum',
                                               'registration_fees': 'sum',
                                               'travel_accommodation': 'sum',
                                               'fees': 'sum',
                                               'related_expenses': 'sum',
                                               'total': 'sum'
                                              })


    grouped = grouped.sort_values(['year', 'total'], ascending=False)
    
    grouped.drop(remcols, axis=1, inplace=True)

    write_to_excel(grouped, out_path + 'excel/' + out + '.xlsx', open=open_sheet, index=False)
    grouped.to_csv(out_path + 'csv/' + out + '.csv', index=False)
    
#HCP
"""
aggregate_by_recipient(df_hcp[df_hcp.year == YEAR].copy(), groupby=['group', 'year'], out='%s_hcp_recipients_aggregated' % YEAR, remcols = [])
aggregate_by_recipient(df_hcp.copy(), groupby=['group', 'year'], out='all_hcp_recipients_each_year', remcols = [])
aggregate_by_recipient(df_hcp.copy(), groupby=['group'], out='all_hcp_recipients_sum_all_years', remcols = ['year'])
"""
   
#HCO
#aggregate_by_recipient(df_hco[df_hco.year == YEAR].copy(), groupby=['group', 'year'], out='%s_hco_recipients_aggregated' % YEAR, remcols = [])
#aggregate_by_recipient(df_hco.copy(), groupby=['group', 'year'], out='all_hco_recipients_each_year', remcols = [])
aggregate_by_recipient(df_hco.copy(), groupby=['group'], out='all_hco_recipients_sum_all_years', remcols = ['year'])
